In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
#import matplotlib.pyplot as plt
import time
import os
import copy
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler
from train_snippets import train_model
import ipdb
#from visualize_images import imshow
import time


In [ ]:
import logging
logging.basicConfig(filename='train{}.log'.format(), level=logging.INFO, filemode='w', format='%(asctime)s %(levelname)-8s %(message)s',
                              datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}
mask_tensor = mask_tensor = torch.Tensor([
        [1,0,0],
        [1,0,1],
        [0,1,1],
        [0,1,0],
        [0,0,1]
    ])



In [3]:
dataset = ImageFolder('/home/hthieu/AICityChallenge2019/data/Track2Data/vehicle_views_v3/', transform=data_transforms['train'],
                     target_transform=lambda x: mask_tensor[x])
print(dataset)

Dataset ImageFolder
    Number of datapoints: 15343
    Root Location: /home/hthieu/AICityChallenge2019/data/Track2Data/vehicle_views_v3/
    Transforms (if any): Compose(
                             RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): <function <lambda> at 0x7f3978f837b8>


In [4]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
val_dataset.dataset = copy.copy(dataset) #copy, so it can have other transformation.
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

dataloaders = {}
dataloaders['train'] = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
dataloaders['val'] = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8)

In [5]:
logging.info(dataloaders['train'].dataset)
logging.info('*'*20)
logging.info(dataloaders['val'].dataset)
logging.info('logged')

In [6]:
N_CLASSES = 3

model_ft = models.resnet50(pretrained=True)

ct = 0
for child in model_ft.children():
    ct += 1
    if ct <= 7:
        print('freezing',child)
        for param in child.parameters():
            param.requires_grad = False
    else:
        print(child,'remains unfrozen')

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, N_CLASSES)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

criterion = nn.BCEWithLogitsLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters())

# Decay LR by a factor of 0.1 every 10 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft,mode='min',factor=0.1,patience=10,threshold=0.0001)

freezing Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
freezing BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
freezing ReLU(inplace)
freezing MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
freezing Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, lr_scheduler,dataloaders=dataloaders,
                       num_epochs=100, device=device)

Epoch 0/99
----------
train Loss: 0.1732 Acc: 0.9319
val Loss: 0.1218 Acc: 0.9645
saving checkpoint as best model

Epoch 1/99
----------
train Loss: 0.1234 Acc: 0.9536
val Loss: 0.0618 Acc: 0.9785
saving checkpoint as best model

Epoch 2/99
----------
train Loss: 0.1106 Acc: 0.9581
val Loss: 0.0525 Acc: 0.9834
saving checkpoint as best model

Epoch 3/99
----------
train Loss: 0.0983 Acc: 0.9632
val Loss: 0.0541 Acc: 0.9831

Epoch 4/99
----------
train Loss: 0.0851 Acc: 0.9695
val Loss: 0.0472 Acc: 0.9836
saving checkpoint as best model

Epoch 5/99
----------
train Loss: 0.0866 Acc: 0.9664
val Loss: 0.0400 Acc: 0.9867
saving checkpoint as best model

Epoch 6/99
----------
train Loss: 0.0823 Acc: 0.9693
val Loss: 0.0428 Acc: 0.9877
saving checkpoint as best model

Epoch 7/99
----------
train Loss: 0.0735 Acc: 0.9724
val Loss: 0.0479 Acc: 0.9854

Epoch 8/99
----------
train Loss: 0.0740 Acc: 0.9727
val Loss: 0.0419 Acc: 0.9864

Epoch 9/99
----------
train Loss: 0.0684 Acc: 0.9736
val Loss

In [30]:
best_ckpt = torch.load('./model_best.pth.tar')

In [32]:
best_ckpt['best_acc']

tensor(0.9909, device='cuda:0')

# Test

In [36]:
best = torch.load('./best_model_03_05_resnet50_0.9909/model_best.pth.tar')
model_ft.load_state_dict(best['state_dict'])
model_ft.eval()
print(best['best_acc'])
print(best['epoch'])

tensor(0.9909, device='cuda:0')
77


In [ ]:
print(model_ft)

In [14]:
model = model_ft
model.eval()   # Set model to evaluate mode
running_loss = 0.0
running_corrects = 0
dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}
predicts = []

running_corrects = []
i=0
for inputs, labels in dataloaders['val']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    i+=1
    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs)
        preds = (outputs >= 0.5).float()
        running_corrects.append(torch.sum(preds == labels.data, dim=0))
    # statistics


running_corrects = torch.stack(running_corrects,dim=0)
epoch_acc = torch.mean(torch.sum(running_corrects,dim=0).float() / float(dataset_sizes['val'])) #mean acc

In [74]:
inputs, _ = val_dataset[6000]
classes = torch.sigmoid(model_ft(torch.unsqueeze(inputs,dim=0).cuda()))>=0.5
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[str(x) for x in classes])

IndexError: index 6000 is out of bounds for dimension 0 with size 3069

In [62]:
torch.unsqueeze(inputs,dim=0).shape

torch.Size([1, 3, 224, 224])

In [118]:
dts_test = ImageFolder('/home/bnminh/projects/ai2/datasets/ai_dataset/DATA/image_test_root/',transform=data_transforms['val'],
                     target_transform=lambda x: mask_tensor[x])
dtl_test = DataLoader(dts_test,batch_size=64, shuffle=False)

dts_test.imgs

outputs_accumulated = []
for inputs, labels in dtl_test:
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs)
        outputs_accumulated.append(outputs)
    # statistics

outputs_accumulated = torch.cat(outputs_accumulated,dim=0)

outputs_accumulated.shape

In [122]:
outputs_accumulated_np = outputs_accumulated.cpu().numpy()

np.savetxt('test_view_score.txt',outputs_accumulated_np,fmt="%.4f")

# test best imgs

In [5]:
from PIL import Image
class CSVDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.csv = np.genfromtxt(csv_file, delimiter=',', dtype='str')

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.csv[idx][1])
        
        image = Image.open(img_name)
        
       
        if self.transform:
            sample = self.transform(image)

        return sample

In [6]:
dts_test_best = CSVDataset(csv_file='/home/bnminh/projects/ai2/SOURCE/view_cls/track2_test_best_imgs.txt',
                          root_dir='/home/bnminh/projects/ai2/datasets/ai_dataset/DATA/image_train/',
                          transform=data_transforms['val'])

In [7]:
len(dts_test_best)

798

In [8]:
dtl_test_best = DataLoader(dts_test_best,batch_size=64, shuffle=False)

In [9]:
best = torch.load('./model_best.pth.tar')
model_ft.load_state_dict(best['state_dict'])
model_ft.eval()
print(best['best_acc'])
print(best['epoch'])

outputs_accumulated = []
for inputs in dtl_test_best:
    inputs = inputs.to(device)
    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        outputs = torch.sigmoid(outputs)
        outputs_accumulated.append(outputs)
    # statistics

tensor(0.9646, device='cuda:0')
83


In [13]:
torch.sum(outputs_accumulated[1])

tensor(1.0418, device='cuda:0')

In [10]:
outputs_accumulated = torch.cat(outputs_accumulated,dim=0)
outputs_accumulated_np = outputs_accumulated.cpu().numpy()

In [11]:
np.savetxt('test_best_view_score_fixed.txt',outputs_accumulated_np,fmt="%.4f")

In [15]:
dtl_test_best.dataset.csv

array([['1', '000029.jpg', '0'],
       ['1', '016762.jpg', '1'],
       ['1', '000105.jpg', '2'],
       ...,
       ['1', '018160.jpg', '795'],
       ['1', '014521.jpg', '796'],
       ['1', '015490.jpg', '797']], dtype='<U10')

In [65]:
12*64

768

# Query

In [5]:
best = torch.load('./model_best.pth.tar')
model_ft.load_state_dict(best['state_dict'])
model_ft.eval()
print(best['best_acc'])
print(best['epoch'])

dts_qr = ImageFolder('/home/bnminh/projects/ai2/datasets/ai_dataset/DATA/image_query_root/',transform=data_transforms['val'],
                     target_transform=lambda x: mask_tensor[x])
dtl_qr = DataLoader(dts_qr,batch_size=64, shuffle=False)

dts_qr.imgs

outputs_accumulated = []
for inputs, labels in dtl_qr:
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        outputs = torch.sigmoid(outputs)
        outputs_accumulated.append(outputs)
    # statistics

outputs_accumulated = torch.cat(outputs_accumulated,dim=0)

outputs_accumulated.shape

tensor(0.9646, device='cuda:0')
83


torch.Size([1052, 3])

In [8]:
outputs_accumulated_np = outputs_accumulated.cpu().numpy()
np.savetxt('query_view_score.txt',outputs_accumulated_np,fmt="%.4f")

In [10]:
len(dts_qr.imgs)

1052

In [7]:
outputs_accumulated

tensor([[9.8239e-02, 9.2024e-01, 2.5852e-03],
        [5.6148e-03, 4.1065e-01, 9.6782e-01],
        [9.9914e-01, 4.1695e-05, 4.1868e-03],
        ...,
        [9.6563e-01, 3.2542e-02, 7.7149e-03],
        [5.7503e-02, 5.0148e-02, 9.8266e-01],
        [9.9882e-01, 7.6558e-04, 1.2866e-06]], device='cuda:0')